# Search for proteins with keywords close to the studied protein and whose neighbors resemble the neighbors of this same protein

- Enter the name of the studied protein

In [ ]:
target="FAM111B"
top=10
main_keyword=[]

## Library

In [ ]:
%pip install goatools
%pip install colorama
from goatools import obo_parser
import requests
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
import networkx as nx
import json
from scipy.spatial import distance
from colorama import Back, Style, Fore
from google.colab import drive
from google.colab import files
drive.mount('/content/gdrive')


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## Execution

In [ ]:
go = obo_parser.GODag("/content/gdrive/MyDrive/Proximity_keywords/go-basic.obo")

/content/gdrive/MyDrive/Proximity_keywords/go-basic.obo: fmt(1.2) rel(2022-03-22) 47,103 Terms


In [ ]:
with open("/content/gdrive/MyDrive/Proximity_keywords/bp_string2.json","r") as f:
    dico_bp=json.load(f)


In [ ]:
with open("/content/gdrive/MyDrive/Proximity_keywords/kw_string3.json", "r") as f:
        dico_kw=json.load(f)
len(dico_kw.keys())

16854

In [ ]:
with open("/content/gdrive/MyDrive/Proximity_keywords/dico_mlb.json", "r") as f:
        dico_mlb=json.load(f)
len(dico_mlb.keys())

16854

In [ ]:
#Create a dict with protein name as keys and keywords as values
dico_mlb_kw={}
for k in dico_mlb.keys():
  array=np.zeros((1,789))
  for i in dico_mlb[k][0]:
    array[0,int(i)]=1
  dico_mlb_kw[k]=array

In [ ]:
#Create a dict with protein name as keys and the keywords of their neighbors as values
dico_mlb_neig={}
for k in dico_mlb.keys():
  array=np.zeros((1,789))
  for i in dico_mlb[k][1]:
    array[0,int(i)]=1
  dico_mlb_neig[k]=array


In [ ]:
list_unique_kw=[]
for i in list(dico_kw.values()):
  for j in i :
    if j not in list_unique_kw:
      list_unique_kw.append(j)
len(list_unique_kw)
main_keyword in list_unique_kw

False

In [ ]:
# Calculate the Hamming distance between the keywords of the target and the keywords of each proteins
dico_h_kw={}
for k in dico_mlb.keys():
            if k!=target:
              if len(set(main_keyword).intersection(set(list_unique_kw))) >= len(main_keyword):
                if len(set(main_keyword).intersection(set(dico_kw[k])))>=len(main_keyword):
                  h1=distance.hamming(dico_mlb_kw[k],dico_mlb_kw[target])
                  dico_h_kw[k]=h1
        
              else:
                  h1=distance.hamming(dico_mlb_kw[k],dico_mlb_kw[target])
                  dico_h_kw[k]=h1

len(dico_h_kw.keys())

16853

In [ ]:
len(set(main_keyword).intersection(set(dico_kw["ABHD12B"])))>=len(main_keyword)
len(list(dico_h_kw.keys()))

16853

In [ ]:
# Calculate the Hamming distance between the keywords of the neighbors of the target and the keywords of the neighbors of each proteins
dico_h_neig={}
for k in dico_h_kw.keys():
            if k!=target:
                h1=distance.hamming(dico_mlb_neig[k],dico_mlb_neig[target])
                dico_h_neig[k]=h1
len(dico_h_neig.keys())

16853

In [ ]:
# Select the closest
top_kw=np.argsort(list(dico_h_kw.values()))[:top]

In [ ]:
list_top=[]
for i in top_kw:
    list_top.append(list(dico_h_kw.keys())[i])

In [ ]:
for i in np.argsort([dico_h_kw[j] for j in list_top]):
  print(list_top[i]," : ",dico_h_kw[list_top[i]])

C19orf68  :  0.005069708491761723
MACROD1  :  0.005069708491761723
C19orf47  :  0.0063371356147021544
MPPED1  :  0.0063371356147021544
C6orf222  :  0.0063371356147021544
GPATCH4  :  0.0063371356147021544
LIN37  :  0.0063371356147021544
ABHD12B  :  0.0063371356147021544
FNBP4  :  0.0063371356147021544
C5orf24  :  0.0063371356147021544


In [ ]:
list_d=[]
for i in list_top:
  list_d.append(dico_h_neig[i])

In [ ]:
list_top_final=[]
for i in np.argsort(list_d):
  list_top_final.append(list_top[i])
  print(list_top[i]," : ",list_d[i])

ABHD12B  :  0.032953105196451206
C6orf222  :  0.034220532319391636
C19orf68  :  0.044359949302915085
MPPED1  :  0.07984790874524715
C5orf24  :  0.08238276299112801
FNBP4  :  0.08871989860583017
C19orf47  :  0.09252217997465145
LIN37  :  0.10139416983523447
GPATCH4  :  0.12927756653992395
MACROD1  :  0.13307984790874525


## Results

In [ ]:
with open("proximity_kw.txt","w") as f:
  if target not in dico_kw.keys():
    print("This protein is not in the dataset")
  else:
    kw_target=dico_kw[target]
    print("*"*50,"\n")
    f.write("*"*50)
    f.write("\n")
    print(target,"\n",dico_kw[target])
    f.write(target)
    f.write("\n")
    f.write(f"{dico_kw[target]}")

    for i in range(len(list_top_final)):
      print("\n","*"*50,"\n")
      f.write("\n")
      f.write("*"*50)
      f.write("\n")
      print(list_top_final[i],"\n")
      f.write(list_top_final[i])
      f.write("\n")
      for n in dico_kw[list_top_final[i]]:
        if n not in kw_target:
          print(Fore.WHITE + Back.RED + f"{n}", end=" ")
          f.write(n)
          print(Style.RESET_ALL, end=" ")
        else:
          print(n,end=" ")
          f.write(n)
      print("\n")
      f.write("\n")
      print([go[y].name for y in dico_bp[list_top_final[i]]])
      f.write(f"{[go[y].name for y in dico_bp[list_top_final[i]]]}")
    files.download("proximity_kw.txt")


************************************************** 

FAM111B 
 ['Acetylation', 'Alternative splicing', 'Disease variant', 'Hydrolase', 'Isopeptide bond', 'Protease', 'Reference proteome', 'Ubl conjugation']

 ************************************************** 

ABHD12B 

Alternative splicing Hydrolase Reference proteome 

['monoacylglycerol catabolic process', 'phosphatidylserine catabolic process', 'macromolecule depalmitoylation']

 ************************************************** 

C6orf222 

Isopeptide bond Reference proteome Ubl conjugation 

[]

 ************************************************** 

C19orf68 

Alternative splicing Isopeptide bond Reference proteome Ubl conjugation 

[]

 ************************************************** 

MPPED1 

Alternative splicing Hydrolase Reference proteome 

['biological_process']

 ************************************************** 

C5orf24 

Alternative splicing Isopeptide bond Phosphoprotein  Reference proteome Ubl conjugation 

[]



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>